In [ ]:
import numpy as np 
import pandas as pd

: 

In [ ]:
# Chargement des fichiers CSV dans des DataFrames

df_train = pd.read_csv('train.csv')
df_test  = pd.read_csv('test.csv')
df_struct = pd.read_csv('structures.csv')

# Extraction des types uniques dans df_test
unique_type = list(set(df_test['type']))

elements = []
# Récupération des 3e et 4e caractères de chaque type unique pour identifier les éléments
for i in unique_type: 
    elements.append(i[2])
    elements.append(i[3])
unique_elements = list(set(elements))  # Eléments uniques

: 

In [ ]:
# Fonction pour donner un ID unique à chaque type
def give_uniqueId(lis): 
    l = np.size(lis)
    fin = np.zeros((l,1))
    for i in range(0,l):
        fin[i] = (unique_type.index(lis[i])+1)
    return fin

# Fonction pour donner un ID unique à chaque élément
def give_uniqueElement(lis):
    element = np.frompyfunc(lambda x:x[3:4],1,1)(lis)
    uniqueID = np.frompyfunc(lambda x:unique_elements.index(x)-1,1,1)(element)
    return uniqueID

# Fonction pour extraire le premier caractère de la chaîne de type et le transformer en entier
def giveFirst(lis):
    return np.frompyfunc(lambda x:int(x[0:1])-1,1,1)(lis)

# Fonction pour fusionner les informations d'atome dans le DataFrame
def map_atom_info(df, atom_idx):
    df = pd.merge(df, df_struct, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

: 

In [ ]:
# Fusion des valeurs x, y, z dans les DataFrames train et test
train_df = map_atom_info(map_atom_info(df_train,0),1)
test_df = map_atom_info(map_atom_info(df_test,0),1)

: 

In [ ]:
# Premier paramètre du type
train_df['typeval']= giveFirst(train_df['type'].values)
test_df['typeval']= giveFirst(test_df['type'].values)

# Donne l'avant-dernier élément comme paramètre (le premier élément est toujours l'hydrogène)
train_df['elem'] = give_uniqueElement(train_df['type'].values)
test_df['elem']  = give_uniqueElement(test_df['type'].values)

# Attribution de valeurs numériques uniques au type
train_df['type'] = give_uniqueId(train_df['type'].values)
test_df['type'] = give_uniqueId(test_df['type'].values)

: 

In [ ]:
train_df.head(4)

: 

In [ ]:
test_df.head(3)

: 

In [ ]:
train_df['dx'] = train_df['x_0']-train_df['x_1']
train_df['dy'] = train_df['y_0']-train_df['y_1']
train_df['dz'] = train_df['z_0']-train_df['z_1']
test_df['dx'] = test_df['x_0']-test_df['x_1']
test_df['dy'] = test_df['y_0']-test_df['y_1']
test_df['dz'] = test_df['z_0']-test_df['z_1']

: 

In [ ]:
train_df.head(4)

: 

In [ ]:
# Attribution des caractéristiques et des étiquettes
X_train = train_df[['x_0','y_0','z_0','x_1','y_1','z_1','dx','dy','dz','elem','typeval','type']].values
y_train = train_df['scalar_coupling_constant'].values
X_test = test_df[['x_0','y_0','z_0','x_1','y_1','z_1','dx','dy','dz','elem','typeval','type']].values

: 

In [ ]:
X_train

: 

In [ ]:
# Prétraitement : mise à l'échelle des caractéristiques pour que leur moyenne soit 0
from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

: 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

: 

In [ ]:
# Fonction pour créer et entraîner un réseau de neurones profond (DNN)
def DNN(num_layers, train_X, train_y):
    # Création du modèle
    model = Sequential()
    # Nombre de colonnes dans les données d'entraînement
    n_cols = train_X.shape[1]
    # num_layers = [200, 30, 15, 10]
    # Couche d'entrée avec 200 neurones
    model.add(Dense(num_layers[0], activation='relu', input_shape=(n_cols,)))
    
    # Couches cachées et couche de sortie
    # Index 1,2 : de 1 à len(num_layers)-2
    for i in range(1, len(num_layers) - 1):
        model.add(Dense(num_layers[i], activation='relu'))
    # Couche de sortie
    model.add(Dense(num_layers[-1], activation='linear'))
    # Compilation du modèle avec l'optimiseur adamax et la fonction de perte mean_squared_error
    model.compile(optimizer='adamax', loss='mean_squared_error')
    
    # Définition du moniteur d'arrêt précoce pour arrêter l'entraînement lorsque l'amélioration cesse
    early_stopping_monitor = EarlyStopping(patience=5)
    # Entraînement du modèle
    model.fit(train_X, train_y, 
              validation_split=0.1, epochs=10, callbacks=[early_stopping_monitor])
    return model

: 

In [ ]:
model = DNN([10000,1200,500,300,200,100,1],X_train,y_train)
pred = model.predict(X_test)

: 

In [ ]:
def submit(predictions):
    submit = pd.read_csv('sample_submission.csv')
    print(len(submit), len(predictions))   
    submit["scalar_coupling_constant"] = predictions
    submit.to_csv("submission.csv", index=False)

: 

In [ ]:
submit(pred)

: 